# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [1]:
import pandas as pd
import logging
import json

from utilities import text_to_json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [2]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        responses = scoring_func(inputString)
        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Overwriting driver.py


Let's test the module.

In [3]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [4]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [5]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

'calulating length in javascript. below is my object in javascript, how can i calculate the length of it.   treeobj .length is not working. any help would be appreciated. thanks.'

Here, we convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [6]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [7]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 262.38 ms


In [8]:
resp = predict_for(body)

INFO:model_driver:Predictions: [(5223, 6700, 0.7397060379835241), (4616202, 4616273, 0.6256758901283033), (11922383, 11922384, 0.103836658977746), (4968406, 4968448, 0.06727070788341889), (126100, 4889658, 0.06396938921761833), (3127429, 3127440, 0.026284147276043023), (4935632, 4935684, 0.02182863724181103), (23667086, 23667087, 0.021069930145792767), (85992, 86014, 0.020517213648905516), (14220321, 14220323, 0.01926239528458793), (7837456, 14853974, 0.016277325475266974), (1789945, 1789952, 0.010639583368380391), (2631001, 2631198, 0.010588248696354356), (1458633, 3439981, 0.00994386648079301), (12953704, 12953750, 0.00993171267711581), (750486, 750506, 0.008872452902550959), (2851404, 2851413, 0.008503666729900836), (149055, 149150, 0.007944366472676535), (5431351, 5431435, 0.007271689138468306), (14028959, 8716680, 0.007166692657365668), (728360, 728694, 0.006660239302826183), (111102, 111111, 0.0059137381772876845), (307179, 307200, 0.005622669814086744), (4255472, 4255480, 0.0049

Next, we move on to [building our docker image](03_Build_Image.ipynb).